In [1]:
import zipfile
import os
import pandas as pd
import numpy as np
import cv2 
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Model
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.applications.xception import preprocess_input

2023-06-14 17:17:05.874997: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
# unzipping the main dataset folder 
dataset_zip = "./dogs-vs-cats.zip"
dataset = "./dogs-vs-cats"

if not os.path.exists(dataset):
    os.makedirs(dataset)

with zipfile.ZipFile(dataset_zip,'r') as zip_file:
    zip_file.extractall(dataset)

In [5]:
# unzipping folder with training images
train_data_zip = "./dogs-vs-cats/train.zip"

with zipfile.ZipFile(train_data_zip,'r') as zip_file:
    zip_file.extractall(dataset)

In [6]:
# unzipping folder with test images
test_data_zip = "./dogs-vs-cats/test1.zip"

with zipfile.ZipFile(test_data_zip,'r') as zip_file:
    zip_file.extractall(dataset)

In [9]:
# Labelling training data based on the file name of the image
# creating a dataframe with 2 columns: image file name and the corresponding label

training_data = dataset + "/train"

training_images = os.listdir(training_data)

category = [image.split(".")[0] for image in training_images]

train_df = pd.DataFrame({"Image": training_images,"Category":category})

train_df = train_df[:500]
train_df.head()


,Image,Category
0,dog.8011.jpg,dog
1,cat.5077.jpg,cat
2,dog.7322.jpg,dog
3,cat.2718.jpg,cat
4,cat.10151.jpg,cat


In [62]:
test_data = dataset + "/test1"

test_images = os.listdir(test_data)

test_df = pd.DataFrame({"Image": test_images})

test_df.head()

,Image
0,9733.jpg
1,63.jpg
2,6400.jpg
3,823.jpg
4,4217.jpg


In [63]:
# checking if the value of category is either cat or dog 

train_df.Category.value_counts()

dog    257
cat    243
Name: Category, dtype: int64

In [64]:
# splitting training set into train and validation sets
train, val = train_test_split(train_df, test_size=0.2)

In [65]:
# checking the min and max width and height of the images in the training set
# in order to decide the optimum height and width the images have to be resized to

# Uncomment the following lines to execute

# min_height =  min_width = 2000
# max_height = max_width = 0
# for img in os.listdir(training_data):
#     img = cv2.imread(os.path.join(training_data,img))
#     height = img.shape[0]
#     min_height = min(min_height,height)
#     max_height = max(max_height,height)
#     width = img.shape[1]
#     min_width = min(min_width,width)
#     max_width = max(max_width,width)

# print("Max height: ", max_height)
# print("Min height: ", min_height)
# print("Max width: ", max_width)
# print("Min width: ", min_width)

In [67]:
# ImageDataGenerators for train and validation sets so as to send them as input to the keras model
target_size = 300
batch_size = 32
train_aug_map = image.ImageDataGenerator(
                rotation_range=10,
                horizontal_flip = True,
                fill_mode = 'nearest',
                preprocessing_function = preprocess_input)

train_data = train_aug_map.flow_from_dataframe(
                            train, training_data,
                            x_col='Image',
                            y_col = 'Category',
                            target_size = (target_size, target_size),
                            batch_size= batch_size,
                            class_mode='categorical')


val_aug_mag = image.ImageDataGenerator(
                preprocessing_function = preprocess_input)

val_data = val_aug_mag.flow_from_dataframe(
                            val, training_data,
                            x_col='Image',
                            y_col = 'Category',
                            target_size = (target_size, target_size),
                            batch_size= batch_size,
                            class_mode='categorical')



Found 400 validated image filenames belonging to 2 classes.
Found 100 validated image filenames belonging to 2 classes.


In [69]:
# Image Generator for test suite 

# test_aug_map = image.ImageDataGenerator(
#                 preprocessing_function = preprocess_input)

# test_datagen = test_aug_map.flow_from_dataframe(
#                             test_df[:3], test_data,
#                             x_col='Image',
#                             y_col = None,
#                             class_mode = None,
#                             target_size = (target_size, target_size),
#                             shuffle = False)

Found 3 validated image filenames.


In [33]:
# Stacking layers to get a model that can be trained
# Base layers are taken from Keras' Xception model architecture and top layers are added to it 
base_model = Xception(
    include_top=False,
    weights="imagenet",
    input_shape=(300, 300, 3)
)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
output_layer = Dense(2, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output_layer)

for layer in base_model.layers:
    layer.trainable = False
    
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics = ['accuracy'])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 300, 300, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 149, 149, 32  864         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 149, 149, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                           

 block4_sepconv1_bn (BatchNorma  (None, 37, 37, 728)  2912       ['block4_sepconv1[0][0]']        
 lization)                                                                                        
                                                                                                  
 block4_sepconv2_act (Activatio  (None, 37, 37, 728)  0          ['block4_sepconv1_bn[0][0]']     
 n)                                                                                               
                                                                                                  
 block4_sepconv2 (SeparableConv  (None, 37, 37, 728)  536536     ['block4_sepconv2_act[0][0]']    
 2D)                                                                                              
                                                                                                  
 block4_sepconv2_bn (BatchNorma  (None, 37, 37, 728)  2912       ['block4_sepconv2[0][0]']        
 lization)

 n)                                                                                               
                                                                                                  
 block7_sepconv1 (SeparableConv  (None, 19, 19, 728)  536536     ['block7_sepconv1_act[0][0]']    
 2D)                                                                                              
                                                                                                  
 block7_sepconv1_bn (BatchNorma  (None, 19, 19, 728)  2912       ['block7_sepconv1[0][0]']        
 lization)                                                                                        
                                                                                                  
 block7_sepconv2_act (Activatio  (None, 19, 19, 728)  0          ['block7_sepconv1_bn[0][0]']     
 n)                                                                                               
          

 block9_sepconv3_bn (BatchNorma  (None, 19, 19, 728)  2912       ['block9_sepconv3[0][0]']        
 lization)                                                                                        
                                                                                                  
 add_19 (Add)                   (None, 19, 19, 728)  0           ['block9_sepconv3_bn[0][0]',     
                                                                  'add_18[0][0]']                 
                                                                                                  
 block10_sepconv1_act (Activati  (None, 19, 19, 728)  0          ['add_19[0][0]']                 
 on)                                                                                              
                                                                                                  
 block10_sepconv1 (SeparableCon  (None, 19, 19, 728)  536536     ['block10_sepconv1_act[0][0]']   
 v2D)     

                                                                                                  
 block12_sepconv3_act (Activati  (None, 19, 19, 728)  0          ['block12_sepconv2_bn[0][0]']    
 on)                                                                                              
                                                                                                  
 block12_sepconv3 (SeparableCon  (None, 19, 19, 728)  536536     ['block12_sepconv3_act[0][0]']   
 v2D)                                                                                             
                                                                                                  
 block12_sepconv3_bn (BatchNorm  (None, 19, 19, 728)  2912       ['block12_sepconv3[0][0]']       
 alization)                                                                                       
                                                                                                  
 add_22 (A

In [88]:
# Training the model and saving the trained model for future use
train_images = train.shape[0]
val_images = val.shape[0]
model.fit(train_data, epochs=2,validation_data = val_data, 
                    validation_steps=val_images//batch_size, 
                    steps_per_epoch=train_images//batch_size)
model.save("img_cls_model.h5")

Epoch 1/2


2023-06-09 18:46:02.990514: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


12/12 [==============================] - ETA: 0s - loss: 0.0015 - accuracy: 1.0000

2023-06-09 18:47:13.351630: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


12/12 [==============================] - 87s 8s/step - loss: 0.0015 - accuracy: 1.0000 - val_loss: 0.0044 - val_accuracy: 1.0000
Epoch 2/2
12/12 [==============================] - 71s 6s/step - loss: 1.0594e-04 - accuracy: 1.0000 - val_loss: 0.0020 - val_accuracy: 1.0000


In [35]:
# Function to preprocess images before sending them to the model for prediction
def preprocess(image_path):
    img = image.load_img(image_path, target_size=(300,300))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

In [89]:
# Since the target labels were categorical, the model converted them to numerical for training 
# class_indices attribute contains the labels along with the numerical mapping that was used for training
labels = dict((v,k) for k,v in train_data.class_indices.items())
def find_label(prediction):
    return labels[prediction]

{0: 'cat', 1: 'dog'}


In [86]:
# predicting image using trained model
img = test_data + "/5.jpg"
input_img = preprocess(img)
prediction = np.argmax(model.predict(input_img))
print(find_label(prediction))

1/1 [==============================] - 0s 173ms/step
cat
